# 🧪 Class Imbalance Strategies - CTI-BERT ile Adım Adım Test

Her stratejiyi **bağımsız** olarak test et, sonuçları karşılaştır.

## 🤖 Model: CTI-BERT (IBM Research)
- **ibm-research/CTI-BERT**: Cyber Threat Intelligence verisiyle önceden eğitilmiş domain-specific BERT
- **Avantaj**: Güvenlik ve CTI metinlerini anlamada genel BERT'ten daha iyi
- **Reference**: https://huggingface.co/ibm-research/CTI-BERT

## 📊 Dataset: Single Source
- **tumeteor/Security-TTP-Mapping** (14.9k train + 2.6k test)
- **Özellik**: MITRE ATT&CK technique ID'leri (T-codes)
- **Avantaj**: Tutarlı label format, yüksek kalite

## 💡 Kullanım:
1. Setup hücrelerini çalıştır (1-4)
2. İstediğin stratejiyi seç (Strategy 1-5)
3. Sadece o stratejiyi çalıştır (~30-45 dk)
4. Results Comparison hücresini çalıştır
5. Başka bir strateji test et

**Avantaj:** Her stratejiyi ayrı ayrı test edip sonuçları görebilirsin!


## 🔧 SETUP (Her zaman çalıştır)

In [ ]:
# Update repository to latest version (get optimized tree classifiers)
!cd /content/Mitre_Attack_TTP_Mapping && git pull origin main

print("\n⚡ OPTIMIZATION APPLIED:")
print("   - ExtraTreesClassifier: 50 trees (was 100)")
print("   - RandomForestClassifier: 50 trees (was 100)")
print("   - max_features='sqrt' (~28 features per split instead of 768)")
print("   - min_samples_split=20 (faster splits)")
print("   - Expected speedup: ~4x faster training!")
print("\n   If training was stuck, Runtime > Interrupt execution, then re-run from Strategy cell\n")

### 1. Colab Setup

In [ ]:
import sys
import os

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("✅ Google Colab ortamı tespit edildi")
    
    import torch
    if torch.cuda.is_available():
        print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
        print(f"   VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    else:
        print("⚠️  GPU bulunamadı! Runtime > Change runtime type > GPU seçin")
    
    print("\n📥 Proje indiriliyor...")
    !rm -rf Mitre_Attack_TTP_Mapping
    !git clone https://github.com/Aliekinozcetin/Mitre_Attack_TTP_Mapping.git
    os.chdir('Mitre_Attack_TTP_Mapping')
    print(f"✅ Çalışma dizini: {os.getcwd()}")
    
    print("\n📦 Paketler yükleniyor...")
    !pip install -q torch transformers datasets scikit-learn pandas tqdm matplotlib seaborn
    print("✅ Tüm paketler yüklendi")
    
    # HuggingFace bağlantı optimizasyonu
    print("\n🔧 HuggingFace cache ayarları...")
    
    # Create cache directory
    cache_dir = '/content/hf_cache'
    os.makedirs(cache_dir, exist_ok=True)
    
    # Set environment variables
    os.environ['HF_HOME'] = cache_dir
    os.environ['TRANSFORMERS_CACHE'] = cache_dir
    os.environ['HF_DATASETS_CACHE'] = cache_dir
    os.environ['HF_HUB_DOWNLOAD_TIMEOUT'] = '600'  # 10 minutes
    os.environ['CURL_CA_BUNDLE'] = ''
    os.environ['HF_ENDPOINT'] = 'https://huggingface.co'
    os.environ['HF_HUB_ENABLE_HF_TRANSFER'] = '1'  # Faster downloads
    
    print(f"✅ Cache dizini oluşturuldu: {cache_dir}")
    print(f"   Timeout: 10 dakika")
    
    # Test HuggingFace connection
    try:
        from huggingface_hub import HfApi
        api = HfApi()
        print("\n📡 HuggingFace bağlantı testi...")
        info = api.model_info("ibm-research/CTI-BERT", timeout=30)
        print(f"✅ Model erişilebilir: {info.modelId}")
    except Exception as e:
        print(f"⚠️  Bağlantı uyarısı: {str(e)[:100]}")
        print("   Model indirme denemeye devam edilecek...")
else:
    print("ℹ️  Yerel ortamda çalışıyorsunuz")

### 2. Import Modules

In [ ]:
# Clear import cache
import sys
if 'src.data_loader' in sys.modules:
    del sys.modules['src.data_loader']
if 'src.model' in sys.modules:
    del sys.modules['src.model']
if 'src.train' in sys.modules:
    del sys.modules['src.train']
if 'src.evaluate' in sys.modules:
    del sys.modules['src.evaluate']
if 'src.strategies' in sys.modules:
    del sys.modules['src.strategies']

import torch
import numpy as np
import json
from datetime import datetime
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

# Import prepare_data first
from src.data_loader import prepare_data

# CRITICAL FIX: Define the function HERE if import fails
try:
    from src.data_loader import load_datasets_and_prepare_dataloaders
    print("✅ Fonksiyon GitHub'dan import edildi")
except ImportError:
    print("⚠️  GitHub import başarısız, fonksiyon notebook'ta tanımlanıyor...")
    
    def load_datasets_and_prepare_dataloaders(
        model_name: str = "ibm-research/CTI-BERT",
        batch_size: int = 16,
        max_length: int = 512,
        use_hybrid: bool = True,
        dataset_name: str = "tumeteor/Security-TTP-Mapping"
    ):
        """Wrapper for prepare_data - notebook fallback version."""
        data = prepare_data(
            model_name=model_name,
            max_length=max_length,
            use_hybrid=use_hybrid,
            dataset_name=dataset_name
        )
        
        train_loader = DataLoader(
            data['train_dataset'],
            batch_size=batch_size,
            shuffle=True
        )
        
        return (
            train_loader,
            None,
            data['test_dataset'],
            data['label_list']
        )
    
    print("✅ Fallback fonksiyon tanımlandı (CTI-BERT)")

from src.model import load_model
from src.train import train_model
from src.evaluate import evaluate_model
from src.strategies import get_strategy_config

print("✅ Modüller yüklendi")
print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")


---

## 🔧 CONFIGURATION

In [ ]:
# Base training configuration
BASE_CONFIG = {
    'model_name': 'ibm-research/CTI-BERT',  # CTI domain-specific BERT
    'batch_size': 16,
    'learning_rate': 2e-5,
    'num_epochs': 3,
    'max_length': 128,
    'device': 'cuda' if torch.cuda.is_available() else 'cpu'
}

# Output directory
OUTPUT_DIR = Path('outputs')
OUTPUT_DIR.mkdir(exist_ok=True)

# Store results from all strategies
all_test_results = {}

print("✅ Konfigürasyon ayarlandı")
print(f"Model: {BASE_CONFIG['model_name']}")
print(f"Device: {BASE_CONFIG['device']}")
print(f"Output: {OUTPUT_DIR.absolute()}")


---

## 📊 DATA LOADING

In [ ]:
print("📥 Veri yükleniyor...")
print("📦 Dataset: tumeteor/Security-TTP-Mapping (Single Source)")
print(f"🤖 Model: {BASE_CONFIG['model_name']}")
print("")

# Use single dataset: tumeteor only
train_dataloader, val_dataloader, test_dataset, label_names = load_datasets_and_prepare_dataloaders(
    model_name=BASE_CONFIG['model_name'],
    batch_size=BASE_CONFIG['batch_size'],
    max_length=BASE_CONFIG['max_length'],
    use_hybrid=False,  # Single dataset: tumeteor only
    dataset_name="tumeteor/Security-TTP-Mapping"
)

# Get train_dataset from dataloader for strategies
train_dataset = train_dataloader.dataset

# Create test dataloader
test_dataloader = DataLoader(
    test_dataset,
    batch_size=BASE_CONFIG['batch_size'],
    shuffle=False
)

num_labels = len(label_names)

# Create data dict for backward compatibility
data = {
    'train_dataset': train_dataset,
    'test_dataset': test_dataset,
    'label_list': label_names,
    'num_labels': num_labels
}

print(f"✅ Veri yüklendi")
print(f"   Train batches: {len(train_dataloader)}")
print(f"   Test batches: {len(test_dataloader)}")
print(f"   Toplam label sayısı: {num_labels}")
print(f"   İlk 5 label: {label_names[:5]}")


---

## 🧪 STRATEGY TESTING

Her strateji bağımsız olarak test edilebilir. İstediğiniz hücreyi çalıştırın.

### Strategy 1: Baseline (Standard BCE Loss)

**Açıklama:** Standart Binary Cross-Entropy loss kullanır. Referans performans için baseline.

**Önerilen Kullanım:** İlk olarak bu stratejiyi çalıştırın (30-45 dakika).

In [ ]:
strategy_name = "baseline"
print(f"\n{'='*60}")
print(f"🧪 STRATEGY 1: Baseline BCE")
print(f"{'='*60}\n")

# Get strategy configuration
strategy_config = get_strategy_config(
    strategy_name=strategy_name,
    train_dataset=data['train_dataset'],
    num_labels=num_labels,
    label_list=label_names
)

# Create DataLoader
if strategy_config['custom_dataloader'] is not None:
    strategy_train_dataloader = strategy_config['custom_dataloader'](BASE_CONFIG['batch_size'])
else:
    strategy_train_dataloader = DataLoader(
        strategy_config['dataset'],
        batch_size=BASE_CONFIG['batch_size'],
        shuffle=True
    )

print("📋 Konfigürasyon:")
print(f"   Strategy: {strategy_config['name']}")
print(f"   Description: {strategy_config['description']}")
print(f"   Num labels: {strategy_config['num_labels']}")
print(f"   Focal loss: {strategy_config['use_focal_loss']}")
if strategy_config['pos_weight'] is not None:
    print(f"   Pos weight: min={strategy_config['pos_weight'].min():.2f}, max={strategy_config['pos_weight'].max():.2f}")

# Load model
print("\n🔧 Model yükleniyor...")
model = load_model(
    model_name=BASE_CONFIG['model_name'],
    num_labels=strategy_config['num_labels'],
    device=BASE_CONFIG['device'],
    use_focal_loss=strategy_config['use_focal_loss'],
    focal_alpha=strategy_config.get('focal_alpha', 0.25),
    focal_gamma=strategy_config.get('focal_gamma', 2.0),
    pos_weight=strategy_config['pos_weight']
)

# Train model
print("\n🚀 Eğitim başlıyor...")
training_history = train_model(
    model=model,
    train_dataloader=strategy_train_dataloader,
    num_epochs=BASE_CONFIG['num_epochs'],
    learning_rate=BASE_CONFIG['learning_rate'],
    device=BASE_CONFIG['device']
)

# Evaluate model
print("\n📊 Test seti değerlendiriliyor...")
test_results = evaluate_model(
    model=model,
    test_dataset=data['test_dataset'],
    batch_size=BASE_CONFIG['batch_size'],
    device=BASE_CONFIG['device']
)

# Store results
all_test_results[strategy_name] = {
    'config': strategy_config['name'],
    'description': strategy_config['description'],
    'results': test_results
}

# Display results
print("\n" + "="*60)
print(f"✅ STRATEGY 1 TAMAMLANDI: {strategy_name}")
print("="*60)
print(f"\n📈 Sonuçlar:")
for metric, value in test_results.items():
    if isinstance(value, (int, float)):
        print(f"   {metric}: {value:.4f}")
print("\n")

### Strategy 2: Weighted BCE Loss

**Açıklama:** Her label için frekans bazlı ağırlık hesaplar (pos_weight=458 for rare labels). Class imbalance için en etkili yöntem.

**Önerilen Kullanım:** Baseline'dan sonra bu stratejiyi test edin. F1 > 0.15 ise, diğer stratejileri test etmeye gerek yok.

In [ ]:
strategy_name = "weighted"
print(f"\n{'='*60}")
print(f"🧪 STRATEGY 2: Weighted BCE")
print(f"{'='*60}\n")

# Get strategy configuration
strategy_config = get_strategy_config(
    strategy_name=strategy_name,
    train_dataset=data['train_dataset'],
    num_labels=num_labels,
    label_list=label_names
)

# Create DataLoader
if strategy_config['custom_dataloader'] is not None:
    strategy_train_dataloader = strategy_config['custom_dataloader'](BASE_CONFIG['batch_size'])
else:
    strategy_train_dataloader = DataLoader(
        strategy_config['dataset'],
        batch_size=BASE_CONFIG['batch_size'],
        shuffle=True
    )

print("📋 Konfigürasyon:")
print(f"   Strategy: {strategy_config['name']}")
print(f"   Description: {strategy_config['description']}")
print(f"   Num labels: {strategy_config['num_labels']}")
print(f"   Focal loss: {strategy_config['use_focal_loss']}")
if strategy_config['pos_weight'] is not None:
    print(f"   Pos weight: min={strategy_config['pos_weight'].min():.2f}, max={strategy_config['pos_weight'].max():.2f}")

# Load model
print("\n🔧 Model yükleniyor...")
model = load_model(
    model_name=BASE_CONFIG['model_name'],
    num_labels=strategy_config['num_labels'],
    device=BASE_CONFIG['device'],
    use_focal_loss=strategy_config['use_focal_loss'],
    focal_alpha=strategy_config.get('focal_alpha', 0.25),
    focal_gamma=strategy_config.get('focal_gamma', 2.0),
    pos_weight=strategy_config['pos_weight']
)

# Train model
print("\n🚀 Eğitim başlıyor...")
training_history = train_model(
    model=model,
    train_dataloader=strategy_train_dataloader,
    num_epochs=BASE_CONFIG['num_epochs'],
    learning_rate=BASE_CONFIG['learning_rate'],
    device=BASE_CONFIG['device']
)

# Evaluate model
print("\n📊 Test seti değerlendiriliyor...")
test_results = evaluate_model(
    model=model,
    test_dataset=data['test_dataset'],
    batch_size=BASE_CONFIG['batch_size'],
    device=BASE_CONFIG['device']
)

# Store results
all_test_results[strategy_name] = {
    'config': strategy_config['name'],
    'description': strategy_config['description'],
    'results': test_results
}

# Display results
print("\n" + "="*60)
print(f"✅ STRATEGY 2 TAMAMLANDI: {strategy_name}")
print("="*60)
print(f"\n📈 Sonuçlar:")
for metric, value in test_results.items():
    if isinstance(value, (int, float)):
        print(f"   {metric}: {value:.4f}")
print("\n")

### Strategy 3: Focal Loss (γ=2, α=0.25)

**Açıklama:** Focal Loss with moderate focusing (γ=2). Hard örneklere odaklanır.

**Önerilen Kullanım:** Weighted BCE başarısız olursa deneyin.

In [ ]:
strategy_name = "focal_weak"
print(f"\n{'='*60}")
print(f"🧪 STRATEGY 3: Focal Loss (γ=2)")
print(f"{'='*60}\n")

# Get strategy configuration
strategy_config = get_strategy_config(
    strategy_name=strategy_name,
    train_dataset=data['train_dataset'],
    num_labels=num_labels,
    label_list=label_names
)

# Create DataLoader
if strategy_config['custom_dataloader'] is not None:
    strategy_train_dataloader = strategy_config['custom_dataloader'](BASE_CONFIG['batch_size'])
else:
    strategy_train_dataloader = DataLoader(
        strategy_config['dataset'],
        batch_size=BASE_CONFIG['batch_size'],
        shuffle=True
    )

print("📋 Konfigürasyon:")
print(f"   Strategy: {strategy_config['name']}")
print(f"   Description: {strategy_config['description']}")
print(f"   Num labels: {strategy_config['num_labels']}")
print(f"   Focal loss: {strategy_config['use_focal_loss']}")
if strategy_config['use_focal_loss']:
    print(f"   Focal alpha: {strategy_config.get('focal_alpha')}")
    print(f"   Focal gamma: {strategy_config.get('focal_gamma')}")

# Load model
print("\n🔧 Model yükleniyor...")
model = load_model(
    model_name=BASE_CONFIG['model_name'],
    num_labels=strategy_config['num_labels'],
    device=BASE_CONFIG['device'],
    use_focal_loss=strategy_config['use_focal_loss'],
    focal_alpha=strategy_config.get('focal_alpha', 0.25),
    focal_gamma=strategy_config.get('focal_gamma', 2.0),
    pos_weight=strategy_config['pos_weight']
)

# Train model
print("\n🚀 Eğitim başlıyor...")
training_history = train_model(
    model=model,
    train_dataloader=strategy_train_dataloader,
    num_epochs=BASE_CONFIG['num_epochs'],
    learning_rate=BASE_CONFIG['learning_rate'],
    device=BASE_CONFIG['device']
)

# Evaluate model
print("\n📊 Test seti değerlendiriliyor...")
test_results = evaluate_model(
    model=model,
    test_dataset=data['test_dataset'],
    batch_size=BASE_CONFIG['batch_size'],
    device=BASE_CONFIG['device']
)

# Store results
all_test_results[strategy_name] = {
    'config': strategy_config['name'],
    'description': strategy_config['description'],
    'results': test_results
}

# Display results
print("\n" + "="*60)
print(f"✅ STRATEGY 3 TAMAMLANDI: {strategy_name}")
print("="*60)
print(f"\n📈 Sonuçlar:")
for metric, value in test_results.items():
    if isinstance(value, (int, float)):
        print(f"   {metric}: {value:.4f}")
print("\n")

### Strategy 4: Focal Loss (γ=5, α=0.25)

**Açıklama:** Focal Loss with strong focusing (γ=5). Çok hard örneklere odaklanır.

**Önerilen Kullanım:** Focal γ=2 başarılı olursa daha güçlü versiyon için deneyin.

In [ ]:
strategy_name = "focal_strong"
print(f"\n{'='*60}")
print(f"🧪 STRATEGY 4: Focal Loss (γ=5)")
print(f"{'='*60}\n")

# Get strategy configuration
strategy_config = get_strategy_config(
    strategy_name=strategy_name,
    train_dataset=data['train_dataset'],
    num_labels=num_labels,
    label_list=label_names
)

# Create DataLoader
if strategy_config['custom_dataloader'] is not None:
    strategy_train_dataloader = strategy_config['custom_dataloader'](BASE_CONFIG['batch_size'])
else:
    strategy_train_dataloader = DataLoader(
        strategy_config['dataset'],
        batch_size=BASE_CONFIG['batch_size'],
        shuffle=True
    )

print("📋 Konfigürasyon:")
print(f"   Strategy: {strategy_config['name']}")
print(f"   Description: {strategy_config['description']}")
print(f"   Num labels: {strategy_config['num_labels']}")
print(f"   Focal loss: {strategy_config['use_focal_loss']}")
if strategy_config['use_focal_loss']:
    print(f"   Focal alpha: {strategy_config.get('focal_alpha')}")
    print(f"   Focal gamma: {strategy_config.get('focal_gamma')}")

# Load model
print("\n🔧 Model yükleniyor...")
model = load_model(
    model_name=BASE_CONFIG['model_name'],
    num_labels=strategy_config['num_labels'],
    device=BASE_CONFIG['device'],
    use_focal_loss=strategy_config['use_focal_loss'],
    focal_alpha=strategy_config.get('focal_alpha', 0.25),
    focal_gamma=strategy_config.get('focal_gamma', 2.0),
    pos_weight=strategy_config['pos_weight']
)

# Train model
print("\n🚀 Eğitim başlıyor...")
training_history = train_model(
    model=model,
    train_dataloader=strategy_train_dataloader,
    num_epochs=BASE_CONFIG['num_epochs'],
    learning_rate=BASE_CONFIG['learning_rate'],
    device=BASE_CONFIG['device']
)

# Evaluate model
print("\n📊 Test seti değerlendiriliyor...")
test_results = evaluate_model(
    model=model,
    test_dataset=data['test_dataset'],
    batch_size=BASE_CONFIG['batch_size'],
    device=BASE_CONFIG['device']
)

# Store results
all_test_results[strategy_name] = {
    'config': strategy_config['name'],
    'description': strategy_config['description'],
    'results': test_results
}

# Display results
print("\n" + "="*60)
print(f"✅ STRATEGY 4 TAMAMLANDI: {strategy_name}")
print("="*60)
print(f"\n📈 Sonuçlar:")
for metric, value in test_results.items():
    if isinstance(value, (int, float)):
        print(f"   {metric}: {value:.4f}")
print("\n")

### Strategy 5: Top-K Label Analysis (Capacity Test)

**Açıklama:** 5 farklı K değeri ile model kapasitesini test eder. Her K için baseline BCE kullanır.

**Test Edilen K Değerleri:**
- Top-100: Geniş label seti
- Top-50: Orta seviye
- Top-20: Küçük label seti
- Top-10: Minimal label seti
- Top-5: En küçük subset

**Çıktılar:**
- CSV dosyası: Tüm metriklerin karşılaştırması
- 4 grafikli histogram: F1 vs K, Hamming Loss vs K, Recall@5, All Metrics

**Önerilen Kullanım:** Model'in farklı label sayılarındaki performansını görmek için çalıştırın (~2-2.5 saat).

In [ ]:
# Top-K Label Analysis: Test model capacity with different label subset sizes
print(f"\n{'='*60}")
print(f"🧪 TOP-K LABEL ANALYSIS")
print(f"{'='*60}\n")

from src.strategies import filter_top_k_labels

# Test different K values
k_values = [100, 50, 20, 10, 5]
topk_results = {}

for k in k_values:
    print(f"\n{'='*60}")
    print(f"🔬 Testing Top-{k} Labels")
    print(f"{'='*60}\n")
    
    # Filter dataset to top-k labels
    filtered_train_ds, filtered_label_list, label_mapping = filter_top_k_labels(
        data['train_dataset'], 
        label_names, 
        k=k
    )
    filtered_test_ds, _, _ = filter_top_k_labels(
        data['test_dataset'], 
        label_names, 
        k=k
    )
    
    print(f"📊 Dataset Statistics:")
    print(f"   Top-{k} labels selected")
    print(f"   Train samples: {len(filtered_train_ds)}")
    print(f"   Test samples: {len(filtered_test_ds)}")
    print(f"   Labels: {filtered_label_list[:5]}...")
    
    # Create dataloaders
    topk_train_loader = DataLoader(
        filtered_train_ds,
        batch_size=BASE_CONFIG['batch_size'],
        shuffle=True
    )
    topk_test_loader = DataLoader(
        filtered_test_ds,
        batch_size=BASE_CONFIG['batch_size'],
        shuffle=False
    )
    
    # Load model for this K
    print(f"\n🔧 Loading model for {k} labels...")
    topk_model = load_model(
        model_name=BASE_CONFIG['model_name'],
        num_labels=k,
        device=BASE_CONFIG['device'],
        use_focal_loss=False,
        pos_weight=None
    )
    
    # Train model
    print(f"\n🚀 Training on Top-{k}...")
    topk_history = train_model(
        model=topk_model,
        train_dataloader=topk_train_loader,
        num_epochs=BASE_CONFIG['num_epochs'],
        learning_rate=BASE_CONFIG['learning_rate'],
        device=BASE_CONFIG['device']
    )
    
    # Evaluate model
    print(f"\n📊 Evaluating Top-{k}...")
    topk_test_results = evaluate_model(
        model=topk_model,
        test_dataloader=topk_test_loader,
        label_names=filtered_label_list,
        device=BASE_CONFIG['device']
    )
    
    # Store results
    topk_results[f'top_{k}'] = {
        'k': k,
        'num_train': len(filtered_train_ds),
        'num_test': len(filtered_test_ds),
        'metrics': topk_test_results,
        'labels': filtered_label_list
    }
    
    # Display results
    print(f"\n✅ Top-{k} Results:")
    for metric, value in topk_test_results.items():
        if isinstance(value, (int, float)):
            print(f"   {metric}: {value:.4f}")

# Create comparison DataFrame
print(f"\n{'='*60}")
print(f"📊 TOP-K COMPARISON TABLE")
print(f"{'='*60}\n")

topk_comparison = []
for key, data in topk_results.items():
    metrics = data['metrics']
    topk_comparison.append({
        'K': data['k'],
        'Train Samples': data['num_train'],
        'Test Samples': data['num_test'],
        'Micro F1': metrics.get('micro_f1', 0),
        'Hamming Loss': metrics.get('hamming_loss', 0),
        'Micro Precision': metrics.get('micro_precision', 0),
        'Micro Recall': metrics.get('micro_recall', 0),
        'Recall@5': metrics.get('recall_at_5', 0),
        'Precision@5': metrics.get('precision_at_5', 0)
    })

df_topk = pd.DataFrame(topk_comparison)
df_topk = df_topk.sort_values('K', ascending=False)
print(df_topk.to_string(index=False))

# Save CSV
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
topk_csv = OUTPUT_DIR / f'topk_analysis_{timestamp}.csv'
df_topk.to_csv(topk_csv, index=False)
print(f"\n✅ CSV saved: {topk_csv}")

# Create visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: F1 Score vs K
ax = axes[0, 0]
ax.plot(df_topk['K'], df_topk['Micro F1'], marker='o', linewidth=2, markersize=8, label='Micro F1', color='blue')
ax.set_xlabel('Number of Labels (K)', fontsize=12)
ax.set_ylabel('Micro F1 Score', fontsize=12)
ax.set_title('Model Performance vs Label Count', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.legend()
ax.invert_xaxis()  # Higher K on left

# Plot 2: Hamming Loss vs K
ax = axes[0, 1]
ax.plot(df_topk['K'], df_topk['Hamming Loss'], marker='s', linewidth=2, markersize=8, label='Hamming Loss', color='red')
ax.set_xlabel('Number of Labels (K)', fontsize=12)
ax.set_ylabel('Hamming Loss (lower is better)', fontsize=12)
ax.set_title('Hamming Loss vs Label Count', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.legend()
ax.invert_xaxis()  # Higher K on left

# Plot 3: Recall@5 vs K
ax = axes[1, 0]
ax.bar(range(len(df_topk)), df_topk['Recall@5'], alpha=0.7, color='coral')
ax.set_xticks(range(len(df_topk)))
ax.set_xticklabels([f'Top-{k}' for k in df_topk['K']])
ax.set_ylabel('Recall@5', fontsize=12)
ax.set_title('Top-5 Recall by Label Count', fontsize=14, fontweight='bold')
ax.grid(axis='y', alpha=0.3)
for i, v in enumerate(df_topk['Recall@5']):
    ax.text(i, v + 0.02, f'{v:.3f}', ha='center', fontsize=10)

# Plot 4: All metrics comparison
ax = axes[1, 1]
x = range(len(df_topk))
width = 0.2
ax.bar([i - width*1.5 for i in x], df_topk['Micro F1'], width, label='F1', alpha=0.8)
ax.bar([i - width*0.5 for i in x], df_topk['Micro Precision'], width, label='Precision', alpha=0.8)
ax.bar([i + width*0.5 for i in x], df_topk['Micro Recall'], width, label='Recall', alpha=0.8)
ax.bar([i + width*1.5 for i in x], df_topk['Recall@5'], width, label='Recall@5', alpha=0.8)
ax.set_xticks(x)
ax.set_xticklabels([f'Top-{k}' for k in df_topk['K']])
ax.set_ylabel('Score', fontsize=12)
ax.set_title('All Metrics Comparison', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig(OUTPUT_DIR / f'topk_analysis_{timestamp}.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\n✅ Histogram saved: {OUTPUT_DIR / f'topk_analysis_{timestamp}.png'}")

# Store in all_test_results for comparison
for key, data in topk_results.items():
    all_test_results[key] = {
        'config': f"Top-{data['k']} Labels",
        'description': f"Baseline BCE with {data['k']} most frequent labels",
        'results': data['metrics']
    }

print(f"\n{'='*60}")
print(f"✅ TOP-K ANALYSIS COMPLETE")
print(f"{'='*60}\n")

### Strategy 6: Classifier Chain (Multi-label Classification)

**Açıklama:** BERT embeddings + sklearn ClassifierChain. Her label'ı sırayla tahmin ederken önceki tahminleri feature olarak kullanır (label dependencies).

**Avantajlar:**
- Label arası bağımlılıkları öğrenir
- Binary classification chain (her label için ayrı model)
- Class imbalance her binary problem'de ayrı handle edilir

**Base Estimator:** Logistic Regression (class_weight='balanced')

**Önerilen Kullanım:** Diğer stratejilerle karşılaştırmak için çalıştırın (~45-60 dakika).

In [ ]:
strategy_name = "classifier_chain"
print(f"\n{'='*60}")
print(f"🧪 STRATEGY 6: Classifier Chain")
print(f"{'='*60}\n")

from src.classifier_chain import train_classifier_chain, evaluate_classifier_chain

# Load BERT model for feature extraction (without classification head)
print("🔧 Loading BERT model for embeddings...")
bert_model_for_chain = load_model(
    model_name=BASE_CONFIG['model_name'],
    num_labels=num_labels,
    device=BASE_CONFIG['device'],
    use_focal_loss=False,
    pos_weight=None
)

print("📋 Konfigürasyon:")
print(f"   Strategy: Classifier Chain")
print(f"   Description: BERT embeddings + sklearn ClassifierChain")
print(f"   Base estimator: LogisticRegression (balanced)")
print(f"   Chain order: random")
print(f"   Label dependencies: Yes (chain structure)")

# Train ClassifierChain
print("\n🔗 Training Classifier Chain...")
chain_model = train_classifier_chain(
    bert_model=bert_model_for_chain,
    train_dataloader=train_dataloader,
    device=BASE_CONFIG['device'],
    base_estimator='logistic',  # 'logistic' or 'random_forest'
    order='random',  # Random order for chain
    cv=None,  # Use true labels (not cross-validated)
    random_state=42
)

# Evaluate model
print("\n📊 Test seti değerlendiriliyor...")
test_results = evaluate_classifier_chain(
    model=chain_model,
    test_dataloader=test_dataloader,
    label_names=label_names
)

# Store results
all_test_results[strategy_name] = {
    'config': 'Classifier Chain (Logistic)',
    'description': 'BERT + ClassifierChain with label dependencies',
    'results': test_results
}

# Display results
print("\n" + "="*60)
print(f"✅ STRATEGY 6 TAMAMLANDI: {strategy_name}")
print("="*60)
print(f"\n📈 Sonuçlar:")
for metric, value in test_results.items():
    if isinstance(value, (int, float)):
        print(f"   {metric}: {value:.4f}")
print("\n")

---

## 🔄 PART C: HYBRID STRATEGIES (Loss × Classification)

**Comprehensive Testing:** 4 Loss Functions × 4 Classification Methods = **16 Kombinasyonlar**

Bu bölüm **tek döngü** ile tüm kombinasyonları test eder:

### Loss Functions:
1. **Baseline BCE** - Standard binary cross-entropy
2. **Weighted BCE** - Frequency-based weights (pos_weight)
3. **Focal Loss γ=2** - Moderate hard example focusing
4. **Focal Loss γ=5** - Strong hard example focusing

### Classification Methods:
1. **ClassifierChain** - Label dependencies (sequential)
2. **MultiOutputClassifier** - Independent labels (parallel)
3. **ExtraTreesClassifier** - Extremely randomized trees (faster, more diverse)
4. **RandomForestClassifier** - Ensemble of decision trees (optimal splits)

### Strategy Matrix:
```
                    ClassifierChain  MultiOutput  ExtraTrees  RandomForest
Baseline BCE        Strategy 7       Strategy 8   Strategy 9  Strategy 10
Weighted BCE        Strategy 11      Strategy 12  Strategy 13 Strategy 14
Focal γ=2           Strategy 15      Strategy 16  Strategy 17 Strategy 18
Focal γ=5           Strategy 19      Strategy 20  Strategy 21 Strategy 22
```

**RandomForest vs ExtraTrees:**
- **RandomForest**: Optimal split selection → slightly better accuracy, slower
- **ExtraTrees**: Random split selection → faster, more diverse, less overfitting
- Both: class_weight='balanced', n_jobs=-1 (parallel training)

**Toplam Süre:** ~12-16 saat (her kombinasyon ~45-60 dakika)

**Kullanım:** Aşağıdaki hücreyi çalıştırarak tüm kombinasyonları otomatik olarak test edin.

In [ ]:
print(f"\n{'='*80}")
print(f"🔄 HYBRID STRATEGIES: LOSS FUNCTIONS × CLASSIFICATION METHODS")
print(f"{'='*80}\n")

from src.classifier_chain import train_classifier_chain, evaluate_classifier_chain
from src.classifier_chain import train_multi_output_classifier, evaluate_multi_output_classifier

# Define loss configurations
# Format: (name, use_focal_loss, pos_weight_config, focal_alpha, focal_gamma)
loss_configs = []

# 1. Baseline BCE
strategy_config_baseline = get_strategy_config(
    strategy_name='baseline',
    train_dataset=data['train_dataset'],
    num_labels=num_labels,
    label_list=label_names
)
loss_configs.append({
    'name': 'baseline',
    'display_name': 'Baseline BCE',
    'use_focal_loss': False,
    'pos_weight': None,
    'focal_alpha': None,
    'focal_gamma': None
})

# 2. Weighted BCE
strategy_config_weighted = get_strategy_config(
    strategy_name='weighted',
    train_dataset=data['train_dataset'],
    num_labels=num_labels,
    label_list=label_names
)
loss_configs.append({
    'name': 'weighted',
    'display_name': 'Weighted BCE',
    'use_focal_loss': False,
    'pos_weight': strategy_config_weighted['pos_weight'],
    'focal_alpha': None,
    'focal_gamma': None
})

# 3. Focal Loss γ=2
loss_configs.append({
    'name': 'focal_gamma2',
    'display_name': 'Focal Loss (γ=2)',
    'use_focal_loss': True,
    'pos_weight': None,
    'focal_alpha': 0.25,
    'focal_gamma': 2.0
})

# 4. Focal Loss γ=5
loss_configs.append({
    'name': 'focal_gamma5',
    'display_name': 'Focal Loss (γ=5)',
    'use_focal_loss': True,
    'pos_weight': None,
    'focal_alpha': 0.25,
    'focal_gamma': 5.0
})

# Define classification methods
classification_methods = [
    {
        'name': 'classifier_chain',
        'display_name': 'ClassifierChain',
        'base_estimator': 'logistic',
        'train_func': train_classifier_chain,
        'eval_func': evaluate_classifier_chain
    },
    {
        'name': 'multi_output',
        'display_name': 'MultiOutputClassifier',
        'base_estimator': 'logistic',
        'train_func': train_multi_output_classifier,
        'eval_func': evaluate_multi_output_classifier
    },
    {
        'name': 'extra_trees',
        'display_name': 'ExtraTreesClassifier',
        'base_estimator': 'extra_trees',
        'train_func': train_multi_output_classifier,  # Uses MultiOutputClassifier with ExtraTrees
        'eval_func': evaluate_multi_output_classifier
    },
    {
        'name': 'random_forest',
        'display_name': 'RandomForestClassifier',
        'base_estimator': 'random_forest',
        'train_func': train_multi_output_classifier,  # Uses MultiOutputClassifier with RandomForest
        'eval_func': evaluate_multi_output_classifier
    }
]

# Counter for strategy numbering (starting from 7)
strategy_counter = 7

# Test all combinations
print(f"🧪 Testing {len(loss_configs)} loss functions × {len(classification_methods)} classification methods")
print(f"   Total combinations: {len(loss_configs) * len(classification_methods)}")
print(f"   Estimated time: ~{len(loss_configs) * len(classification_methods) * 50} minutes")
print(f"\n{'='*80}\n")

for loss_config in loss_configs:
    for clf_method in classification_methods:
        
        strategy_name = f"hybrid_{loss_config['name']}_{clf_method['name']}"
        
        print(f"\n{'='*80}")
        print(f"🧪 STRATEGY {strategy_counter}: {loss_config['display_name']} + {clf_method['display_name']}")
        print(f"{'='*80}\n")
        
        # STEP 1: Train BERT with specific loss function
        print(f"📋 Configuration:")
        print(f"   Loss Function: {loss_config['display_name']}")
        print(f"   Classification Method: {clf_method['display_name']}")
        print(f"   Use Focal Loss: {loss_config['use_focal_loss']}")
        if loss_config['pos_weight'] is not None:
            print(f"   Pos Weight: min={loss_config['pos_weight'].min():.2f}, max={loss_config['pos_weight'].max():.2f}")
        if loss_config['use_focal_loss']:
            print(f"   Focal Alpha: {loss_config['focal_alpha']}")
            print(f"   Focal Gamma: {loss_config['focal_gamma']}")
        
        # Load and train BERT model
        print(f"\n🔧 Step 1/3: Training BERT with {loss_config['display_name']}...")
        bert_model = load_model(
            model_name=BASE_CONFIG['model_name'],
            num_labels=num_labels,
            device=BASE_CONFIG['device'],
            use_focal_loss=loss_config['use_focal_loss'],
            focal_alpha=loss_config['focal_alpha'],
            focal_gamma=loss_config['focal_gamma'],
            pos_weight=loss_config['pos_weight']
        )
        
        # Train BERT
        training_history = train_model(
            model=bert_model,
            train_dataloader=train_dataloader,
            num_epochs=BASE_CONFIG['num_epochs'],
            learning_rate=BASE_CONFIG['learning_rate'],
            device=BASE_CONFIG['device']
        )
        
        # STEP 2: Train classification method on BERT embeddings
        print(f"\n🔧 Step 2/3: Training {clf_method['display_name']} on BERT embeddings...")
        print(f"   Base Estimator: {clf_method['base_estimator']}")
        
        if clf_method['name'] == 'classifier_chain':
            # Train ClassifierChain
            clf_model = train_classifier_chain(
                bert_model=bert_model,
                train_dataloader=train_dataloader,
                device=BASE_CONFIG['device'],
                base_estimator=clf_method['base_estimator'],
                order='random',
                cv=None,
                random_state=42
            )
        else:
            # Train MultiOutputClassifier (or ExtraTrees variant)
            clf_model = train_multi_output_classifier(
                bert_model=bert_model,
                train_dataloader=train_dataloader,
                device=BASE_CONFIG['device'],
                base_estimator=clf_method['base_estimator'],
                n_jobs=-1,
                random_state=42
            )
        
        # STEP 3: Evaluate
        print(f"\n📊 Step 3/3: Evaluating...")
        test_results = clf_method['eval_func'](
            model=clf_model,
            test_dataloader=test_dataloader,
            label_names=label_names
        )
        
        # Store results
        all_test_results[strategy_name] = {
            'config': f"{loss_config['display_name']} + {clf_method['display_name']}",
            'description': f"BERT trained with {loss_config['display_name']}, embeddings classified with {clf_method['display_name']}",
            'results': test_results
        }
        
        # Display results
        print(f"\n{'='*80}")
        print(f"✅ STRATEGY {strategy_counter} TAMAMLANDI: {strategy_name}")
        print(f"{'='*80}")
        print(f"\n📈 Results:")
        for metric, value in test_results.items():
            if isinstance(value, (int, float)):
                print(f"   {metric}: {value:.4f}")
        print(f"\n{'='*80}\n")
        
        strategy_counter += 1

print(f"\n{'='*80}")
print(f"✅ TÜM HYBRID STRATEGIES TAMAMLANDI!")
print(f"{'='*80}")
print(f"\n📊 Total strategies tested: {strategy_counter - 7}")
print(f"💡 Results stored in 'all_test_results' dictionary")
print(f"💡 Run 'Results Comparison' cell to see detailed comparison\n")

---

## 📊 RESULTS COMPARISON

Çalıştırdığınız stratejilerin sonuçlarını karşılaştırmak için bu hücreyi çalıştırın.

In [ ]:
if len(all_test_results) == 0:
    print("⚠️ Henüz hiçbir strateji test edilmedi!")
else:
    print("\n" + "="*80)
    print("📊 STRATEGY COMPARISON RESULTS")
    print("="*80 + "\n")
    
    # Create comparison table
    comparison_data = []
    for strategy_name, data in all_test_results.items():
        results = data['results']
        comparison_data.append({
            'Strategy': strategy_name,
            'Micro F1': results.get('micro_f1', 0),
            'Hamming Loss': results.get('hamming_loss', 0),
            'Recall@5': results.get('recall_at_5', 0),
            'Precision@5': results.get('precision_at_5', 0),
            'Recall@10': results.get('recall_at_10', 0),
            'Precision@10': results.get('precision_at_10', 0),
            'Example Accuracy': results.get('example_based_accuracy', 0)
        })
    
    df_comparison = pd.DataFrame(comparison_data)
    df_comparison = df_comparison.sort_values('Micro F1', ascending=False)
    
    print(df_comparison.to_string(index=False))
    print("\n" + "="*80)
    
    # Find best strategy
    best_strategy = df_comparison.iloc[0]['Strategy']
    best_f1 = df_comparison.iloc[0]['Micro F1']
    best_hamming = df_comparison.iloc[0]['Hamming Loss']
    
    print(f"\n🏆 EN İYİ STRATEJİ: {best_strategy}")
    print(f"   Micro F1 Score: {best_f1:.4f}")
    print(f"   Hamming Loss: {best_hamming:.4f} (lower is better)")
    
    # Compare to baseline if exists
    if 'baseline' in all_test_results:
        baseline_f1 = all_test_results['baseline']['results'].get('micro_f1', 0)
        improvement = ((best_f1 - baseline_f1) / baseline_f1 * 100) if baseline_f1 > 0 else 0
        print(f"   Baseline'a göre iyileştirme: {improvement:+.2f}%")
    
    # Plot comparison
    if len(all_test_results) > 1:
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))
        
        # F1 scores comparison
        strategies = df_comparison['Strategy'].tolist()
        micro_f1 = df_comparison['Micro F1'].tolist()
        recall_at_5 = df_comparison['Recall@5'].tolist()
        
        x = np.arange(len(strategies))
        width = 0.35
        
        axes[0].bar(x - width/2, micro_f1, width, label='Micro F1', alpha=0.8)
        axes[0].bar(x + width/2, recall_at_5, width, label='Recall@5', alpha=0.8)
        axes[0].set_xlabel('Strategy')
        axes[0].set_ylabel('Score')
        axes[0].set_title('Micro F1 & Recall@5 Comparison')
        axes[0].set_xticks(x)
        axes[0].set_xticklabels(strategies, rotation=45, ha='right')
        axes[0].legend()
        axes[0].grid(axis='y', alpha=0.3)
        
        # Hamming Loss comparison
        hamming_loss_vals = df_comparison['Hamming Loss'].tolist()
        
        axes[1].bar(x, hamming_loss_vals, alpha=0.8, color='coral')
        axes[1].set_xlabel('Strategy')
        axes[1].set_ylabel('Hamming Loss (lower is better)')
        axes[1].set_title('Hamming Loss Comparison')
        axes[1].set_xticks(x)
        axes[1].set_xticklabels(strategies, rotation=45, ha='right')
        axes[1].grid(axis='y', alpha=0.3)
        
        plt.tight_layout()
        plt.show()
    
    print("\n")

---

## 💾 SAVE RESULTS

Sonuçları kaydetmek için bu hücreyi çalıştırın.

In [ ]:
if len(all_test_results) == 0:
    print("⚠️ Kaydedilecek sonuç yok!")
else:
    # Helper function to convert numpy arrays to lists
    def convert_to_serializable(obj):
        """Recursively convert numpy arrays to lists for JSON serialization."""
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        elif isinstance(obj, dict):
            return {key: convert_to_serializable(value) for key, value in obj.items()}
        elif isinstance(obj, list):
            return [convert_to_serializable(item) for item in obj]
        elif isinstance(obj, (np.integer, np.floating)):
            return float(obj)
        else:
            return obj
    
    # Save results to JSON
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    results_file = OUTPUT_DIR / f'strategy_comparison_{timestamp}.json'
    
    # Convert results to JSON-serializable format
    serializable_results = convert_to_serializable(all_test_results)
    
    with open(results_file, 'w') as f:
        json.dump(serializable_results, f, indent=2)
    
    print(f"✅ Sonuçlar kaydedildi: {results_file}")
    
    # Save comparison table to CSV
    comparison_data = []
    for strategy_name, data in all_test_results.items():
        results = data['results']
        comparison_data.append({
            'Strategy': strategy_name,
            'Micro_F1': results.get('micro_f1', 0),
            'Recall_at_5': results.get('recall_at_5', 0),
            'Precision_at_5': results.get('precision_at_5', 0),
            'Recall_at_10': results.get('recall_at_10', 0),
            'Precision_at_10': results.get('precision_at_10', 0),
            'Example_Based_Accuracy': results.get('example_based_accuracy', 0)
        })
    
    df_comparison = pd.DataFrame(comparison_data)
    csv_file = OUTPUT_DIR / f'strategy_comparison_{timestamp}.csv'
    df_comparison.to_csv(csv_file, index=False)
    
    print(f"✅ CSV tablosu kaydedildi: {csv_file}")
    
    # Download files if on Colab
    try:
        from google.colab import files
        files.download(str(results_file))
        files.download(str(csv_file))
        print("✅ Dosyalar indirildi")
    except ImportError:
        print("💡 Colab değil, dosyalar sadece kaydedildi")

---

## 📝 NOTLAR VE ÖNERİLER

**Model:**
- **CTI-BERT** (ibm-research/CTI-BERT): Cyber Threat Intelligence'a özel BERT
- Güvenlik metinlerinde genel BERT'ten daha iyi performans
- Pre-trained on security-specific corpus

**Dataset:**
- Single source: tumeteor/Security-TTP-Mapping (14.9k train + 2.6k test)
- 499 unique MITRE ATT&CK technique labels
- Tutarlı label format, yüksek kalite

**Stratejiler:**

**Loss Function Based (BERT end-to-end):**
1. **Baseline BCE** - Referans performans, class imbalance handle yok
2. **Weighted BCE** - En etkili, pos_weight ile rare label'lara odaklanır
3. **Focal Loss (γ=2)** - Hard örnek odaklı, moderate focusing
4. **Focal Loss (γ=5)** - Hard örnek odaklı, strong focusing

**Multi-label Classification Based:**
6. **Classifier Chain** - BERT embeddings + sklearn ClassifierChain
   - Label dependencies öğrenir (her label önceki tahminleri kullanır)
   - Binary classification chain (her label için ayrı LogisticRegression)
   - Class imbalance her binary problem'de ayrı handle edilir
   - Avantaj: Label arası ilişkileri modelleyebilir
   - Dezavantaj: Training süresi daha uzun (499 binary classifier)

**Capacity Analysis:**
5. **Top-K Labels** - Model kapasitesini test et (K=5,10,20,50,100)

**Test Sırası:**
1. **Baseline** - Her zaman ilk olarak çalıştırın (referans performans)
2. **Weighted BCE** - En umut verici strateji, Micro F1 > 0.10 bekleniyor
3. **Classifier Chain** - Label dependency avantajı test et
4. Weighted BCE başarılı olursa → Bitti, kullan!
5. Classifier Chain daha iyiyse → Multi-label yaklaşım kazandı!
6. İkisi de kötüyse → **Focal Loss (γ=2)** dene
7. Hala kötüyse → **Focal Loss (γ=5)** dene
8. Hepsi başarısız olursa → **Top-100** ile model öğrenme kapasitesini kontrol et

**Başarı Kriterleri:**
- Micro F1 > 0.15 → İyi performans
- Recall@5 > 0.30 → Çok iyi performans
- Hamming Loss < 0.10 → İyi performans
- Micro F1 < 0.10 → Strateji işe yaramadı

**Beklenen Performans (Strategy Karşılaştırması):**
```
Strategy          | Micro F1 | Hamming Loss | Recall@5 | Training Time
------------------|----------|--------------|----------|---------------
Baseline BCE      | ~0.00    | ~0.02        | ~0.13    | 30 min
Weighted BCE      | ~0.10    | ~0.12        | ~0.79    | 30 min ⭐
Focal γ=2         | ~0.00    | ~0.02        | ~0.15    | 30 min
Focal γ=5         | ~0.19    | ~0.03        | ~0.50    | 30 min
Classifier Chain  | ???      | ???          | ???      | 45-60 min 🆕
```

**Metrikler:**
- **Micro F1:** Genel performans (threshold-based)
- **Hamming Loss:** Yanlış tahmin edilen label oranı (0=perfect, düşük=iyi)
- **Recall@5:** Top-5 tahmin içinde kaç doğru label var
- **Precision@5:** Top-5 tahminlerin kaçı doğru
- **Example-Based Accuracy:** Tüm label'ların tam eşleşmesi (çok strict)

**ClassifierChain Detayları:**
- **Nasıl çalışır:** 
  1. BERT ile text → embedding (768-dim vector)
  2. Her label için binary classifier (LogisticRegression)
  3. Chain: label_2 tahmini → label_1'in tahminini feature olarak kullanır
  4. 499 label → 499 binary classifier (zincirleme)
  
- **Avantajlar:**
  - Label dependencies modelleyebilir (örn: T1059 varsa T1059.001 olasılığı artar)
  - Her binary problem balanced olarak handle edilir
  - BERT fine-tuning gerektirmez (sadece embedding extraction)
  
- **Dezavantajlar:**
  - Training süresi daha uzun (499 model)
  - Chain order'a bağımlı (random order kullanıyoruz)
  - Memory kullanımı yüksek

**Süre:**
- Strategy 1-4: ~30-45 dakika
- Strategy 6 (Classifier Chain): ~45-60 dakika (embedding extraction + 499 binary classifiers)
- Strategy 5 (Top-K): ~2-2.5 saat (5 model)

**Hatırlatmalar:**
- Her strateji hücresi bağımsızdır, istediğiniz sırada çalıştırabilirsiniz
- Sonuçlar `all_test_results` dictionary'sinde saklanır
- Comparison hücresini dilediğiniz zaman çalıştırıp ara sonuçlara bakabilirsiniz
- CTI-BERT ilk indirilirken cache'lenir, sonraki çalıştırmalar hızlıdır
- Classifier Chain için sklearn yüklü olmalı (Colab'da default olarak var)